In [1]:
from PIL import Image
import numpy as np
import os

def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            label = 1 if subfolder == 'REAL' else 0
            for filename in os.listdir(subfolder_path):
                img_path = os.path.join(subfolder_path, filename)
                print(filename)
                img = Image.open(img_path)
#                 img = img.resize((32, 32))  
                img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Load training and testing data
train_images, train_labels = load_images_from_folder('train')
test_images, test_labels = load_images_from_folder('test')


1000 (10).jpg
1000 (2).jpg
1000 (3).jpg
1000 (4).jpg
1000 (5).jpg
1000 (6).jpg
1000 (7).jpg
1000 (8).jpg
1000 (9).jpg
1000.jpg
1001 (10).jpg
1001 (2).jpg
1001 (3).jpg
1001 (4).jpg
1001 (5).jpg
1001 (6).jpg
1001 (7).jpg
1001 (8).jpg
1001 (9).jpg
1001.jpg
1002 (10).jpg
1002 (2).jpg
1002 (3).jpg
1002 (4).jpg
1002 (5).jpg
1002 (6).jpg
1002 (7).jpg
1002 (8).jpg
1002 (9).jpg
1002.jpg
1003 (10).jpg
1003 (2).jpg
1003 (3).jpg
1003 (4).jpg
1003 (5).jpg
1003 (6).jpg
1003 (7).jpg
1003 (8).jpg
1003 (9).jpg
1003.jpg
1004 (10).jpg
1004 (2).jpg
1004 (3).jpg
1004 (4).jpg
1004 (5).jpg
1004 (6).jpg
1004 (7).jpg
1004 (8).jpg
1004 (9).jpg
1004.jpg
1005 (10).jpg
1005 (2).jpg
1005 (3).jpg
1005 (4).jpg
1005 (5).jpg
1005 (6).jpg
1005 (7).jpg
1005 (8).jpg
1005 (9).jpg
1005.jpg
1006 (10).jpg
1006 (2).jpg
1006 (3).jpg
1006 (4).jpg
1006 (5).jpg
1006 (6).jpg
1006 (7).jpg
1006 (8).jpg
1006 (9).jpg
1006.jpg
1007 (10).jpg
1007 (2).jpg
1007 (3).jpg
1007 (4).jpg
1007 (5).jpg
1007 (6).jpg
1007 (7).jpg
1007 (8).jpg
1007 (

In [2]:
# Flatten images
train_features = train_images.reshape(len(train_images), -1)
test_features = test_images.reshape(len(test_images), -1)


In [5]:
test_features.shape

(20000, 3072)

In [3]:
x = 1000
subset_train_features = np.vstack((train_features[:x] , train_features[-x:]))
subset_train_labels = np.concatenate((train_labels[:x] , train_labels[-x:]))

In [4]:
subset_train_features.shape

(2000, 3072)

In [5]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle
# Defining the parameters grid for GridSearchCV 
param_grid={'C':[0.1,1,10,100], 
            'gamma':[0.0001,0.001,0.1,1], 
            'kernel':['rbf','poly']} 
  
# Creating a support vector classifier 
svc=svm.SVC(verbose=True, probability= True) 
  
# Creating a model using GridSearchCV with the parameters grid 
classifier=GridSearchCV(svc,param_grid, verbose = True)

# Train the Classifier
classifier.fit(train_features, train_labels)
classifier.fit(subset_train_features, subset_train_labels)


# Save the classifier to a file
with open('svm_classifier_updated_gridsearch.pkl', 'wb') as f:
    pickle.dump(classifier, f)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][L

In [9]:
import pickle

# Save the classifier to a file
with open('svm_classifier_updated_gridsearch.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)

In [13]:
loaded_classifier.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [10]:
# Predict labels for test set
predicted_labels = loaded_classifier.predict(test_features)

# Calculate accuracy
accuracy = np.mean(predicted_labels == test_labels)
print(f'Accuracy: {accuracy*100:.2f}%')


Accuracy: 73.68%


In [12]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(test_labels, predicted_labels))



              precision    recall  f1-score   support

           0       0.75      0.72      0.73     10000
           1       0.73      0.76      0.74     10000

    accuracy                           0.74     20000
   macro avg       0.74      0.74      0.74     20000
weighted avg       0.74      0.74      0.74     20000



In [14]:
conf_matrix = confusion_matrix(test_labels, predicted_labels)
print(conf_matrix)

[[7183 2817]
 [2447 7553]]
